In [0]:
# installing libs

!pip install imutils
!pip install imgaug

In [0]:
# this code help us to reupdate get repo. over existed one 
!ls
!rm -r Google-Colaboratory-datasets
!ls

In [0]:
# cloning the dataset to work on it and do augmentation process on it  
!git clone https://github.com/Rawash/Google-Colaboratory-datasets.git

fatal: destination path 'Google-Colaboratory-datasets' already exists and is not an empty directory.


In [0]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense,BatchNormalization
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from keras.utils import to_categorical

import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import imgaug as ia
from imgaug import augmenters as iaa
import imutils

input_shape = 64 
no_classes = 10



Using TensorFlow backend.


In [0]:
# Dataset augmentation
seq = iaa.Sequential([
    iaa.Noop(), # horizontal flips
    # Small gaussian blur with random sigma between 0 and 0.5.
    # But we only blur about 50% of all images.
    iaa.GaussianBlur(sigma=(0, 3)),
    iaa.Sometimes(0.5,
#       iaa.Invert(0.3),
      iaa.Affine(
        rotate=(-5, 5),
        )
    ),
    iaa.Multiply((0.2,2)),
    # Strengthen or weaken the contrast in each image.
    iaa.ContrastNormalization((0.5, 1.5)),
    # Add gaussian noise.
    # For 50% of all images, we sample the noise once per pixel.
    # For the other 50% of all images, we sample the noise per pixel AND
    # channel. This can change the color (not only brightness) of the
    # pixels.
    iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.02*255), per_channel=0.2),
    # Make some images brighter and some darker.
    # In 20% of all cases, we sample the multiplier once per channel,
    # which can end up changing the color of the images.
    # Apply affine transformations to each image.
    # Scale/zoom them, translate/move them, rotate them and shear them.
    iaa.Affine(
        scale={"x": (1, 1.7), "y": (1, 1.7)},

    )
], random_order=True) # apply augmenters in random order

In [0]:

# dataset prebaration

print('collecting dataset') 
numbersPath = glob.glob('./Google-Colaboratory-datasets/dataset/**/*')

stack =[]
def blure(image):
    img = cv2.imread(image,0)
    className = image.split('/')[-2][-1]
    stack.append([img,className]) 

list(map(blure,numbersPath))



X = [item[0] for item in stack]
Y = [item[1] for item in stack]


newX = []
newY = []

for i in range(15):
  augImages = seq.augment_images(X)
  for indx , n in enumerate(augImages):
      newX.append(n)
      newY.append(Y[indx])

for indx , n in enumerate(X):
    newX.append(n)
    newY.append(Y[indx])

    


collecting dataset


In [0]:
# spliting the dataset into testing and training data 

X_train, X_test, y_train, y_test = train_test_split(np.array(newX), np.array(newY), test_size=0.25, random_state=42)

print('dataset gethering finished , the trained is : ',len(X_train))


# reshape for gray channel to (n, 64, 64, 1)
X_train = X_train.reshape(X_train.shape[0], input_shape, input_shape, 1)
X_test = X_test.reshape(X_test.shape[0], input_shape, input_shape, 1)

dataset gethering finished , the trained is :  297600


In [0]:
print(X_train.shape)

(297600, 64, 64, 1)


In [0]:

### Modal architecture.

print('model Init start')

model = Sequential()

model.add(Conv2D(32,(3,3),strides=1,input_shape=(input_shape,input_shape,1),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
          
model.add(Conv2D(64,(3,3),input_shape=(ib nput_shape,input_shape,1),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
          
model.add(Conv2D(128,(3,3),input_shape=(input_shape,input_shape,1),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())

model.add(Dense(units=500,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))
model.add(Dense(units = 250, activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))
model.add(Dense(units = 100, activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))
model.add(Dense(units = no_classes, activation = 'softmax'))

model.summary()


model Init start
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 62, 62, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 29, 29, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 12, 12, 128)       73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 6, 6, 128)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4608)              0   

In [0]:


## code compilation line 
print('compilation start')

model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

print('compilation ended')


compilation start
compilation ended


In [0]:

print('image augmantation process')

parametars = dict(rescale = 1./255,
              zca_whitening = True,
              width_shift_range=0.3,
              shear_range = 0.3,
              zoom_range = 0.35,
              validation_split=0.2,
             )

train_datagen = ImageDataGenerator(**parametars)

validation_datagen = ImageDataGenerator(**parametars)


validation_set = validation_datagen.flow(X_test, y_test,
                                            batch_size = 100)
training_set = train_datagen.flow(X_train, y_train, batch_size = 100)

print('image augmantation process ended')

model.fit_generator(training_set,
                     steps_per_epoch=len(X_train) / 100,
                     epochs=45,
                     validation_data = validation_set,
                     validation_steps = 300)

# train_datagen.fit(images, augment=True, seed=seed)
print('model started saving your prediction model')

image augmantation process


/usr/local/lib/python3.6/dist-packages/keras/preprocessing/image.py:645: UserWarning: This ImageDataGenerator specifies `zca_whitening`, which overrides setting of `featurewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


image augmantation process ended
Epoch 1/45


/usr/local/lib/python3.6/dist-packages/keras/preprocessing/image.py:799: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.6/dist-packages/keras/preprocessing/image.py:817: UserWarning: This ImageDataGenerator specifies `zca_whitening`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


2975/2976 [============================>.] - ETA: 0s - loss: 0.1767 - acc: 0.9450

...

2975/2976 [============================>.] - ETA: 0s - loss: 0.0412 - acc: 0.9857

2976/2976 [==============================] - 266s 89ms/step - loss: 0.0412 - acc: 0.9857 - val_loss: 0.0339 - val_acc: 0.9873
Epoch 8/45
 453/2976 [===>..........................] - ETA: 3:27 - loss: 0.0398 - acc: 0.9861

2975/2976 [============================>.] - ETA: 0s - loss: 0.0394 - acc: 0.9863

2976/2976 [==============================] - 265s 89ms/step - loss: 0.0394 - acc: 0.9863 - val_loss: 0.0322 - val_acc: 0.9878
Epoch 9/45
 452/2976 [===>..........................] - ETA: 3:26 - loss: 0.0400 - acc: 0.9860

1389/2976 [=============>................] - ETA: 2:11 - loss: 0.0395 - acc: 0.9863

In [0]:
# model started saving your prediction model
print('model started saving your prediction model')

model.save('ar_numbers_v6.h5')

 
print('conguratolations you are free to use the classifier :)')

model started saving your prediction model
conguratolations you are free to use the classifier :)


In [0]:
# next code is nessisary for Goolge collab 
from google.colab import files

In [0]:
# next code is nessisary for Goolge collab 
from google.colab import files
# !cd Google-Colaboratory-datasets
!ls
files.download('ar_numbers_v6.h5') 

ar_numbers_v6.h5  datalab  Google-Colaboratory-datasets
